In [1]:
import os
from IPython.display import display

import pandas as pd
import numpy as np

import torch

In [2]:
# check whether your PyTorch has access to GPU
# if you don't, make sure to follow the steps mentioned
#  in the README file.
torch.cuda.is_available()

True

In [3]:
DATA_PATH = os.path.join("data", "career-con-2019")

In [4]:
X_train = pd.read_csv(os.path.join(DATA_PATH, "X_train.csv"))
X_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003386,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [5]:
feature_cols = X_train.columns[3:].values
feature_cols

array(['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W',
       'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z',
       'linear_acceleration_X', 'linear_acceleration_Y',
       'linear_acceleration_Z'], dtype=object)

In [5]:
for series_id, group in X_train.groupby("series_id"):
    print(f"{series_id = }")
    display(group[feature_cols])
    break

series_id = 0


,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.10300,-9.7532
1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003386,0.33995,1.50640,-9.4128
2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.59220,-8.7267
3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.09930,-10.0960
4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.46890,-10.4410
...,...,...,...,...,...,...,...,...,...,...
123,-0.75943,-0.63316,-0.10470,-0.10677,0.041741,0.012857,0.016053,-0.80827,1.79890,-6.8687
124,-0.75945,-0.63313,-0.10473,-0.10683,-0.052041,-0.014752,0.021632,-0.36055,2.35620,-6.8926
125,-0.75949,-0.63309,-0.10467,-0.10690,-0.083083,-0.044480,0.031726,-0.98193,0.97328,-9.9795
126,-0.75950,-0.63307,-0.10464,-0.10693,-0.001305,-0.031012,0.017241,-0.72591,0.34931,-11.9770
